In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("D:/data/Aaron/ProstateMRL/Code/Extraction/PatientKey.csv")

df = df[df["Treatment"] == "20fractions"]
list = df["PatID"].unique().astype(str)
array = np.asarray(list)
print(array)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/data/Aaron/ProstateMRL/Code/Extraction/PatientKey.csv'

In [2]:
import pandas as pd
import numpy as np
import datetime

import sys
import os

current = os.path.dirname(os.path.realpath("Test.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
import ImageFunctions as IF

In [ ]:
# read in patient key
root = UF.DataRoot(2)
LimbusKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey.csv")
AllKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\AllPatientKey_s.csv")

# Loop through treatments
treatments = AllKey["Treatment"].unique()
new_key = pd.DataFrame()
for t in treatments:
    AllKey_t = AllKey[AllKey["Treatment"] == t]
    LimbusKey_t = LimbusKey[LimbusKey["Treatment"] == t]

    # Loop through patients
    patIDs = LimbusKey_t["PatID"].unique()

    for pat in patIDs:
        pat_L = LimbusKey_t[LimbusKey_t["PatID"] == pat]
        pat_A = AllKey_t[AllKey_t["PatID"] == pat]
        print(pat_A)
        print(pat_L)
        # add date and days column from all to limbus key based on Scan
        for i in range(len(pat_L)):
            scan = pat_L.iloc[i]["Scan"]
            fraction = pat_A.iloc[i]["Fraction"]
            date = pat_A[pat_A["Scan"] == scan]["Date"].values[0]
            days = pat_A[pat_A["Scan"] == scan]["Days"].values[0]
            pat_L.at[pat_L.index[i], "Date"] = str(date)
            pat_L.at[pat_L.index[i], "Days"] = int(days)
            pat_L.at[pat_L.index[i], "Fraction"] = int(fraction)

        pat_L["Days"] = pat_L["Days"].astype(int)
        # sort by days
        pat_L = pat_L.sort_values(by=["Fraction"])

    # add to new key
        new_key = new_key.append(pat_L)

new_key.to_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey_s.csv", index=False)


        
        



In [20]:
import pandas as pd
import numpy as np
import os

import sys

current = os.path.dirname(os.path.realpath(''))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
#import ImageFunctions as IF

root = UF.DataRoot(2)

# read in patient key
PatKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\AllPatientKey_s.csv")
PatKey = PatKey[PatKey["Treatment"] == "SABR"]

# read in ft vals
df_all = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\Limbus\\SABR_fts_all.csv")


df_all = df_all.drop(columns = [col for col in df_all.columns if "diagnostics" in col])
df_all = df_all.drop(columns = [col for col in df_all.columns if "Unnamed" in col])

fts = df_all.columns[2:]

fractions = PatKey["Fraction"].unique()
patIDs = PatKey["PatID"].unique()

# merge with patient key to get date and days and fraction
df_all_m = pd.merge(df_all, PatKey[["PatID", "Scan", "Days", "Fraction"]],on = ["PatID", "Scan"], how = "left")

frac = df_all_m.pop("Fraction")
df_all_m.insert(2, "Fraction", frac)

days = df_all_m.pop("Days")
df_all_m.insert(3, "Days", days)


df_all_m["Days"] = df_all_m["Days"].astype(int)
df_all_m["Fraction"] = df_all_m["Fraction"].astype(int)

df_all_m = df_all_m.sort_values(by = ["PatID", "Fraction", "Days"])
df_all_m.to_csv(root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\Limbus\\SABR_fts_all_m.csv", index = False)


